# Image-to-Image Translation with CycleGAN
Author: Jin Yeom (jinyeom@utexas.edu)

![CycleGAN](images/cyclegan.jpg)

In [1]:
import os
import zipfile

import numpy as np
import torch
import wget
from matplotlib import image as mpimg
from matplotlib import pyplot as plt

## Datasets

In [2]:
def download_dataset(name: str):
    if name not in ["ae_photos", "apple2orange", "summer2winter_yosemite", "horse2zebra", 
                    "monet2photo", "cezanne2photo", "ukiyoe2photo", "vangogh2photo", "maps", 
                    "cityscapes", "facades", "iphone2dslr_flower", "ae_photos"]:
        raise ValueError("invalid argument dataset name")
        
    if not os.path.exists("./datasets"):
        print("Datasets directory not found, creating a new directory 'datasets'...")
        os.mkdir("./datasets")
    zip_path = "./datasets/{}.zip".format(name)
    target_dir = "./datasets/{}/".format(name)
    
    url = "https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/{}.zip".format(name)
    wget.download(url, out=zip_path)
    
    os.mkdir(target_dir)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("datasets/")
    os.remove(zip_path)

In [4]:
# NOTE: only download if you have to!
download_dataset("apple2orange")
download_dataset("horse2zebra")

Datasets directory not found, creating a new directory 'datasets'...


In [3]:
!ls ./datasets

apple2orange  horse2zebra


## Model

![model](images/CycleGAN.png)

In [61]:
class ResBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super().__init__()
        self.pad = torch.nn.ReflectionPad2d((kernel_size - 1) // 2)
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.norm1 = torch.nn.InstanceNorm2d(out_channels)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size, stride)
        self.norm2 = torch.nn.InstanceNorm2d(out_channels)

    def forward(self, x):
        z = self.pad(x)
        z = self.conv1(z)
        z = self.norm1(x)
        z = self.relu(z)
        z = self.pad(z)
        z = self.conv2(z)
        z = self.norm2(z)
        return x + z

## Generator network

In this section, we define the architecture of the generator network. Note that the generator network's architecture in CycleGAN is somewhat different from what we're familiar with. Rather than decoding a random Gaussian noise to an image, this generator adopts the mechanism of an image transformation network, which is often used in applications like style transfer and super-resolution. In the origianl paper, 6 residual blocks were used for 128 x 128 images, and 9 for 256 x 256 images. We're going to assume the latter for now, but this can change in the future, based on the desired resolution of the images.

In [2]:
class Generator(torch.nn.Module):
    def __init__(self, in_channel, out_channel):
        self.encoder = torch.nn.Sequential(torch.nn.ReflectionPad2d(3),
                                           torch.nn.Conv2d(in_channel, 32, 7, 1),
                                           torch.nn.InstanceNorm2d(32),
                                           torch.nn.ReLU(),
                                           torch.nn.Conv2d(32, 64, 3, 2),
                                           torch.nn.InstanceNorm2d(64),
                                           torch.nn.ReLU(),
                                           torch.nn.Conv2d(64, 128, 3, 2),
                                           torch.nn.InstanceNorm2d(128),
                                           torch.nn.ReLU())
        self.transformer = torch.nn.Sequential(ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1),
                                               ResBlock(128, 128, 3, 1))
        self.decoder = torch.nn.Sequential(torch.nn.ConvTranspose2d(128, 64, 3, 2),
                                           torch.nn.ReLU(),
                                           torch.nn.InstanceNorm2d(64),
                                           torch.nn.ConvTranspose2d(64, 32, 3, 2),
                                           torch.nn.ReLU(),
                                           torch.nn.ReflectionPad2d(3))
        self.output = torch.nn.Sequential(torch.nn.Conv2d(32, 3, 7, 1),
                                          torch.nn.InstanceNorm2d(3),
                                          torch.nn.Tanh())
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.transformer(x)
        x = self.decoder(x)
        x = self.output(x)
        return x

## Descriminator network

This section describes the descriminator network.

In [18]:
def descriminator(input_, name):
    with tf.variable_scope(name):
        h_0 = tf_conv_block(input_, 64, 7, 2, activation=leaky_relu, normalize=False, name="h_0")
        h_1 = tf_conv_block(h_0, 128, 7, 2, activation=leaky_relu, name="h_1")
        h_2 = tf_conv_block(h_1, 256, 7, 2, activation=leaky_relu, name="h_2")
        h_3 = tf_conv_block(h_2, 512, 7, 2, activation=leaky_relu, name="h_3")
        pred = tf_conv_block(h_3, 1, 7, 1, 1, normalize=False, name="pred")
        return pred

## References
1. https://arxiv.org/pdf/1703.10593.pdf (Orignal CycleGAN paper)
2. https://arxiv.org/pdf/1603.08155v1.pdf (Perceptual losses and image transformation network)
3. https://arxiv.org/pdf/1607.08022.pdf (Instance normalization)
4. https://hardikbansal.github.io/CycleGANBlog/ (TensorFlow tutorial for CycleGAN)